In [ ]:
# Mountain Car- v0 solution using action-value neural network function approximator

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable

# One hidden layer function approximator
class action_value_function(nn.Module):

    def __init__(self):

        super(action_value_function, self).__init__()
        self.block = nn.Sequential(
            torch.nn.Linear(4, 100, bias = False),
#             torch.nn.ReLU(),
#             torch.nn.Linear(20, 4),
#             torch.nn.ReLU(),
            torch.nn.Linear(100, 2, bias = False),
#             torch.nn.Softmax(),
        )

    def forward(self, input):
        return self.block(input)
    
# I tried different weight initializations but found they did not perform well.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, 0, 1)


In [ ]:
# Instaciate action_value_function class
q_hat = action_value_function()
# q_hat.apply(weights_init)

In [ ]:
import gym
import numpy as np

done = False
LEARNING_RATE = 0.1
DISCOUNT = 0.99
EPISODES = 25000
SHOW_EVERY = 300
BATCH_SIZE = 4

env = gym.make("CartPole-v1")
env.reset()

# DISCRETE_OS_SIZE = [20, 20] #* len(env.observation_space.high)
# discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE


In [ ]:
# Test number of actions and other parameters
print('observation_space:',env.observation_space.low,'to',env.observation_space.high,'| Number of action values:',env.action_space.n)

In [ ]:
4.1887903e-01

In [ ]:
# # Function for discretization of observation state space
# def get_discrete_state(state):
#     discrete_state = (state - env.observation_space.low)/discrete_os_win_size
#     return tuple(discrete_state.astype(np.int))  # we use this tuple to look up the 3 Q values for the available actions in the q-table

# # Sampled version of q value table
# q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(q_hat.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
env.observation_space.low

In [ ]:
env.close()
cart_position, cart_velocity, angle, pole_tip_velocity = env.reset()
env.render()
print('cart_position:', cart_position, '| cart_velocit|y:', cart_velocity, '| angle:', angle, '| Pole Tip velocity:', pole_tip_velocity)

In [ ]:
avg_loss = 0
epsilon = 0.5
curr_high_pos = 0
curr_low_pos = 0
successes = 0
STEPS = 200

In [ ]:
# Train agent
import random

for episode in range(EPISODES):
   
    state = torch.from_numpy(env.reset()).float()
    done = False
    ep_reward = 0
    
    if episode%SHOW_EVERY == 0:
        print('#'*50,'| RENDERING |','#'*49)
        RENDER = True
    else:
        RENDER = False

    running_loss = 0
    cont = 0

    for step in range(STEPS):
        if RENDER:
            env.render()
        
        q = q_hat(Variable(state))
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = torch.argmax(q).item()
        
        next_state, reward, done, _ = env.step(action)
        next_state = torch.from_numpy(next_state).float()
        
        
        # zero the parameter gradients
            
#         print(count)
             
        if next_state[0] >= 2.4 or next_state[0] <= -2.4:
            reward -= 1
        if next_state[2] >= 12 or next_state[2] < -12:
            reward -= 1
            
        q_target = q.clone()
        q_target = Variable(q_target.data)
        
        q_target = torch.tensor(reward) + DISCOUNT * q_hat(Variable(next_state))
        # Calculate loss
        loss = criterion(q, q_target)
        optimizer.zero_grad()
        ep_reward += reward
        loss.backward()
    
        optimizer.step()
#         else:
        state = next_state
    
        running_loss += loss.item()

        
    epsilon *= .99
    scheduler.step()

    avg_loss += running_loss
    env.close()
    if episode%100 == 0:
        print('Episode:',episode,'| Average Loss:',avg_loss/100, '| Epsilon:',epsilon, '| Reward:', ep_reward)
        avg_loss = 0
        ep_reward = 0

# env.close()

In [ ]:
# Save model
torch.save(q_hat,'./models/cartpole-v0.pth')

In [ ]:
# Test your policy
env.close()
state = torch.from_numpy(env.reset()).float()
done = False
for step in range(2000):
    env.render()
    q = q_hat(Variable(state))
    action = torch.argmax(q).item()

    next_state, reward, done, _ = env.step(action)
    next_state = torch.from_numpy(next_state).float()
    state = next_state
#     if reward == 1:
#         print('Hell Yes!')
        
    
env.close()